In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import log_loss, brier_score_loss
from sklearn.model_selection import GroupShuffleSplit

from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold

from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
import re
#import torch
#import tensorflow as tf

In [2]:
train_data_df = pd.read_csv('data/trainData.csv')
test_data_df = pd.read_csv('data/testData.csv')

# train_data_df = pd.read_csv('/kaggle/input/data-both/trainData.csv')
# test_data_df = pd.read_csv('/kaggle/input/data-both/testData.csv')

train_data_df.head()

,Race_Time,Race_ID,Course,Distance,distanceYards,Prize,Going,Horse,Trainer,Jockey,...,Speed_2ndPreviousRun,NMFPLTO,MarketOdds_PreviousRun,MarketOdds_2ndPreviousRun,TrainerRating,JockeyRating,daysSinceLastRun,SireRating,DamsireRating,meanRunners
0,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Intervention,Michael Appleby,Aiden Brookes,...,70.0,0.875000,11.03,3.60,2.377268,2.925027,7.0,2.933961,0.467149,10.25
1,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Evocative Spark,Darryll Holland,Christian Howarth,...,48.0,0.181818,42.67,4.19,2.401274,2.611219,13.0,1.934009,0.459547,10.25
2,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Sluzewiec,Scott Dixon,Kieran O'Neill,...,59.0,0.000000,141.13,86.83,2.824967,2.925073,50.0,2.411403,0.456616,8.00
3,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Muscika,David O'Meara,Mark Winn,...,73.0,0.333333,8.97,12.86,2.317504,2.534689,38.0,2.639010,0.462397,10.00
4,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Venturous,David Barron,David Probert,...,62.0,0.090909,44.84,18.20,2.292027,2.448742,24.0,2.494198,0.450770,11.00


In [3]:
known_col_from_train = train_data_df[
    ["betfairSP", "Position", "timeSecs", "pdsBeaten", "NMFP"]
].copy()

train_data_df = train_data_df.drop(columns = ["betfairSP", "Position", "timeSecs", "pdsBeaten", "NMFP"])



known_col_from_test = test_data_df[
    ["betfairSP", "Position", "timeSecs", "pdsBeaten", "NMFP"]
].copy()


test_data_df = test_data_df.drop(columns = ["betfairSP", "Position", "timeSecs", "pdsBeaten", "NMFP"])
known_col_from_test

,betfairSP,Position,timeSecs,pdsBeaten,NMFP
0,140.00,11,137.449997,3.238678,0.000000
1,10.05,8,135.809998,2.641732,0.272727
2,14.15,7,135.759995,2.616483,0.363636
3,58.42,10,137.380005,3.217875,0.090909
4,14.99,3,134.570007,1.681759,0.727273
...,...,...,...,...,...
11270,19.14,6,226.589996,3.589059,0.142857
11271,10.55,7,251.339996,4.977423,0.000000
11272,2.38,3,220.589996,2.282382,0.571429
11273,4.80,2,219.339996,1.458615,0.714286


In [4]:
train_data_df['Win'] = (known_col_from_train['Position'] == 1).astype(int) # Only wins  are kept
test_data_df['Win'] = (known_col_from_test['Position'] == 1).astype(int) # Only wins  are kept

In [5]:
train_data_df_numerical = train_data_df.select_dtypes(exclude='object')
train_data_df_obj = train_data_df.select_dtypes(include='object')

In [6]:
train_data_df.head() # We dropped ["betfairSP", "Position", "timeSecs", "pdsBeaten", "NMFP"] and added ['Win] hence the 5x24
print(train_data_df.shape)

(52099, 24)


In [7]:
train_data_df = train_data_df.drop(columns = ["DamsireRating"])
test_data_df = test_data_df.drop(columns = ["DamsireRating"])

In [8]:
### TRANSFORM
train_data_df['distanceYards_sqrt_new']            = np.sqrt(train_data_df['distanceYards']) # the bins are small
train_data_df['Prize_log_new']                     = np.log1p(train_data_df['Prize'])
train_data_df['MarketOdds_PreviousRun_log_new']    = np.log1p(train_data_df['MarketOdds_PreviousRun'])
train_data_df['MarketOdds_2ndPreviousRun_log_new'] = np.log1p(train_data_df['MarketOdds_2ndPreviousRun'])
train_data_df['daysSinceLastRun_log_new']          = np.log1p(train_data_df['daysSinceLastRun'])

test_data_df['distanceYards_sqrt_new']            = np.sqrt(test_data_df['distanceYards']) # 
test_data_df['Prize_log_new']                     = np.log1p(test_data_df['Prize'])
test_data_df['MarketOdds_PreviousRun_log_new']    = np.log1p(test_data_df['MarketOdds_PreviousRun'])
test_data_df['MarketOdds_2ndPreviousRun_log_new'] = np.log1p(test_data_df['MarketOdds_2ndPreviousRun'])
test_data_df['daysSinceLastRun_log_new']          = np.log1p(test_data_df['daysSinceLastRun'])


print(train_data_df.shape)
print(test_data_df.shape)
#train_data_df = train_data_df.drop(columns = ["distanceYards","Prize", "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun","daysSinceLastRun" ])
#test_data_df = test_data_df.drop(columns = ["distanceYards","Prize", "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun","daysSinceLastRun" ])

print(train_data_df.shape)
print(test_data_df.shape)

(52099, 28)
(11275, 28)
(52099, 28)
(11275, 28)


In [9]:

#df = train_data_df.copy()                    # your dataframe
dt = pd.to_datetime(train_data_df['Race_Time'], dayfirst=True)

train_data_df['Month'] = dt.dt.month
train_data_df['DOW']   = dt.dt.weekday     # 0 = Monday
train_data_df['Hour']  = dt.dt.hour
train_data_df.drop(columns='Race_Time', inplace=True)   # raw string no longer needed


def to_yards(s):
    m = re.search(r'(\d+)m', s)    # miles
    f = re.search(r'(\d+)f', s)    # furlongs
    y = re.search(r'(\d+)y', s)    # yards
    return (
        (int(m.group(1)) * 1760 if m else 0) +
        (int(f.group(1)) * 220  if f else 0) +
        (int(y.group(1))        if y else 0)
    )

train_data_df['DistYards_new'] = train_data_df['Distance'].apply(to_yards)
train_data_df.drop(columns='Distance', inplace=True)   # drop raw text
print(train_data_df.shape)


train_data_df.drop(columns=['Horse'], inplace=True)    # Trainer & Jockey kept for later target-encoding
print(train_data_df.shape)

(52099, 30)
(52099, 29)


In [10]:
#df = test_data_df.copy()                    # your dataframe
dt = pd.to_datetime(test_data_df['Race_Time'], dayfirst=True)

test_data_df['Month'] = dt.dt.month
test_data_df['DOW']   = dt.dt.weekday     # 0 = Monday
test_data_df['Hour']  = dt.dt.hour
test_data_df.drop(columns='Race_Time', inplace=True)   # raw string no longer needed


def to_yards(s):
    m = re.search(r'(\d+)m', s)    # miles
    f = re.search(r'(\d+)f', s)    # furlongs
    y = re.search(r'(\d+)y', s)    # yards
    return (
        (int(m.group(1)) * 1760 if m else 0) +
        (int(f.group(1)) * 220  if f else 0) +
        (int(y.group(1))        if y else 0)
    )

test_data_df['DistYards_new'] = test_data_df['Distance'].apply(to_yards)
test_data_df.drop(columns='Distance', inplace=True)   # drop raw text
print(test_data_df.shape)

#test_data_df.drop(columns=['Horse'], inplace=True)    # Trainer & Jockey kept for later target-encoding -> We dont drop horses either we need it for the probs
print(test_data_df.shape)

(11275, 30)
(11275, 30)


In [11]:
###### ADD NUMERICAL DATA
# could also use MArketOdds_2ndPreviousRun
# I do have log verson already tho
train_data_df['ImpliedProb_prev'] = 1 / train_data_df['MarketOdds_PreviousRun']

# -(best speed = 1) - There rankings compared with each other in a race based on RaceID---
rank_in_field = (
    train_data_df.groupby('Race_ID')['Speed_PreviousRun']
      .rank(method='min', ascending=False)          # 1 = fastest
)

# rescale to 0–1 for numeric stability
train_data_df['SpeedRank'] = (rank_in_field - 1) / (train_data_df['Runners'] - 1)

########################################################################

test_data_df['ImpliedProb_prev'] = 1 / test_data_df['MarketOdds_PreviousRun']

rank_in_field2 = (
    test_data_df.groupby('Race_ID')['Speed_PreviousRun']
      .rank(method='min', ascending=False)          # 1 = fastest
)

# rescale to 0–1 for numeric stability
test_data_df['SpeedRank'] = (rank_in_field2 - 1) / (test_data_df['Runners'] - 1)

In [12]:
ohe_cols    = ['Course', 'Going']
tgt_cols    = ['Trainer', 'Jockey']

num_cols    = ['Runners', 'Age',
               'Speed_PreviousRun', 'Speed_2ndPreviousRun',
               'NMFPLTO', 'TrainerRating', 'JockeyRating',
               'SireRating', 'meanRunners',
               'distanceYards_sqrt_new', 'Prize_log_new',
               'MarketOdds_PreviousRun_log_new',
               'MarketOdds_2ndPreviousRun_log_new',
               'daysSinceLastRun_log_new',
               'Month', 'DOW', 'Hour',
               'DistYards_new', 'ImpliedProb_prev', 'SpeedRank']

drop_cols   = ['Race_ID']  # Bring back later

train_data_df = train_data_df.drop(columns = ["distanceYards","Prize", "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun","daysSinceLastRun" ])
test_data_df = test_data_df.drop(columns = ["distanceYards","Prize", "MarketOdds_PreviousRun","MarketOdds_2ndPreviousRun","daysSinceLastRun" ])

In [13]:
num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median', add_indicator=True)),
    ('scale',  StandardScaler())
])

ohe_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe',    OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

tgt_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('tgt',    TargetEncoder(smoothing=0.3))
])

In [14]:
col_trans = ColumnTransformer([
    ('num', num_pipe,  num_cols),
    ('ohe', ohe_pipe,  ohe_cols),
    ('tgt', tgt_pipe,  tgt_cols)
], remainder='drop')

col_trans

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(add_indicator=True,
                                                                strategy='median')),
                                                 ('scale', StandardScaler())]),
                                 ['Runners', 'Age', 'Speed_PreviousRun',
                                  'Speed_2ndPreviousRun', 'NMFPLTO',
                                  'TrainerRating', 'JockeyRating', 'SireRating',
                                  'meanRunners', 'distanceYards_sqrt_new',
                                  'Prize_log_new',
                                  'MarketOdds_PreviousRun_log_new',
                                  '...
                                  'Hour', 'DistYards_new', 'ImpliedProb_prev',
                                  'SpeedRank']),
                                ('ohe',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Course', 'Going']),
                                ('tgt',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('tgt',
                                                  TargetEncoder(smoothing=0.3))]),
                                 ['Trainer', 'Jockey'])])

In [15]:
main_pipeline = Pipeline([
    ('prep', col_trans),
])

main_pipeline

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Runners', 'Age',
                                                   'Speed_PreviousRun',
                                                   'Speed_2ndPreviousRun',
                                                   'NMFPLTO', 'TrainerRating',
                                                   'JockeyRating', 'SireRating',
                                                   'meanRunners',
                                                   'distanceYards_sqrt_new',
                                                   'Prize_log_new',
                                                   'MarketOdd...
                                                   'DistYards_new',
                                                   'ImpliedProb_prev',
                                                   'SpeedRank']),
                                                 ('ohe',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Course', 'Going']),
                                                 ('tgt',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('tgt',
                                                                   TargetEncoder(smoothing=0.3))]),
                                                  ['Trainer', 'Jockey'])]))])

In [16]:
X = train_data_df.drop(columns=['Race_ID', 'Win'])
y = train_data_df['Win']

In [17]:
X_preproc = main_pipeline.fit_transform(X,y)

In [ ]:
groups = train_data_df['Race_ID']
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_preproc, y, groups))

X_train = X_preproc[train_idx]
X_test = X_preproc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
grp_train = groups.iloc[train_idx]
grp_test = groups.iloc[test_idx]

print("\n=== AFTER FIXING ===")
print(f"Train races: {grp_train.nunique()}")
print(f"Test races: {grp_test.nunique()}")
print(f"Train horses: {len(X_train)}")
print(f"Test horses: {len(X_test)}")
print(f"Train win rate: {y_train.mean():.4f}")
print(f"Test win rate: {y_test.mean():.4f}")

race_overlap = set(grp_train.unique()) & set(grp_test.unique())
if race_overlap:
    print(f"ERROR: {len(race_overlap)} races appear in both train and test!")
    print(f"Overlapping races: {list(race_overlap)[:5]}...")
else:
    print("SUCCESS: No race overlap between train and test")


=== AFTER FIXING ===
Train races: 4269
Test races: 1068
Train horses: 41692
Test horses: 10407
Train win rate: 0.1026
Test win rate: 0.1030
SUCCESS: No race overlap between train and test


In [19]:

# Define LightGBM model
clf = LGBMClassifier(
    learning_rate=0.05,
    n_estimators=500,
    max_depth=3,
    subsample=0.8,
    class_weight='balanced',
    random_state=42,
    verbosity=-1  # <--- suppress warnings and info
)

# Parameter grid
# param_grid = {
#     'n_estimators':  [300, 500,700],
#     'learning_rate': [0.03, 0.05, 0.1],
#     'max_depth':     [2, 3,5]
# }

param_grid = {
    'n_estimators': [900],
    'learning_rate': [0.10800530486428452],
    'max_depth': [7],
    'num_leaves': [135],
    'subsample': [0.8566046529407972],
    'colsample_bytree': [0.8359056649896325],
    'min_child_samples': [18],
    'reg_alpha': [3.619024122032281],
    'reg_lambda': [6.176034056483382],
}

# GroupKFold
gkf = GroupKFold(n_splits=5)

# GridSearchCV
grid_lightgbm = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='neg_log_loss',
    #cv=gkf.split(X_preproc, y, groups),
    cv = gkf.split(X_train, y_train, groups=grp_train),
    n_jobs=-1,
    verbose=0  # <--- suppress GridSearchCV progress bars too
)

# Fit the model
grid_lightgbm.fit(X_train, y_train)

# Output best result
print("Best log-loss :", -grid_lightgbm.best_score_)
best_model = grid_lightgbm.best_estimator_
### Theres this thing called Optuna GOOGLE it maybe can some how decrease idk tho -> Expands grid search biut already takes too long -> Server?

# [I 2025-05-26 17:22:23,134] Trial 30 finished with value: 0.3644336390490304 and parameters: {'n_estimators': 900, 'learning_rate': 0.10800530486428452, 'max_depth': 7, 'num_leaves': 135, 'subsample': 0.8566046529407972, 'colsample_bytree': 0.8359056649896325, 'min_child_samples': 18, 'reg_alpha': 3.619024122032281, 'reg_lambda': 6.176034056483382}. Best is trial 30 with value: 0.3644336390490304.

/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMCla

Best log-loss : 0.3633285443359452


In [ ]:
p_raw = best_model.predict_proba(X_test)[:, 1]

tmp = pd.DataFrame({"Race_ID": grp_test, "p_raw": p_raw})
p_norm = p_raw / tmp.groupby("Race_ID")["p_raw"].transform("sum").to_numpy()

ll = log_loss(y_test,  p_norm)
br = brier_score_loss(y_test, p_norm)

print(f"Hold-out log-loss : {ll:.4f}")
print(f"Hold-out Brier    : {br:.4f}")

/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Hold-out log-loss : 0.3244
Hold-out Brier    : 0.0908


In [ ]:
X_test_raw  = test_data_df.drop(columns=['Race_ID'])  
X_test_pp   = main_pipeline.transform(X_test_raw)                   


p_raw = best_model.predict_proba(X_test_pp)[:, 1]     
pred_df = test_data_df[['Race_ID', 'Horse']].copy()
pred_df['prob_raw'] = p_raw

pred_df['Predicted_Probability'] = (
    pred_df['prob_raw'] /
    pred_df.groupby('Race_ID')['prob_raw'].transform('sum')
)

pred_df = pd.concat([pred_df, known_col_from_test], axis=1)
assert np.allclose(
    pred_df.groupby('Race_ID')['Predicted_Probability'].sum().values, 1.0
)


pred_df.head(19)

/home/ninja123superxl/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Race_ID,Horse,prob_raw,Predicted_Probability,betfairSP,Position,timeSecs,pdsBeaten,NMFP
0,58,Signora Bellissima,0.111151,0.054871,140.00,11,137.449997,3.238678,0.000000
1,58,Sonnerie Power,0.267011,0.131813,10.05,8,135.809998,2.641732,0.272727
2,58,Ribba Hill,0.130881,0.064611,14.15,7,135.759995,2.616483,0.363636
3,58,Orange N Blue,0.010771,0.005317,58.42,10,137.380005,3.217875,0.090909
4,58,Ever Hopefull,0.093416,0.046116,14.99,3,134.570007,1.681759,0.727273
5,58,Percy Willis,0.061726,0.030472,14.23,2,134.320007,1.287854,0.818182
6,58,Hartswood,0.059769,0.029506,29.30,6,135.389999,2.403561,0.454545
7,58,Urban Road,0.099277,0.049010,8.52,4,134.820007,1.963610,0.636364
8,58,Alpine Sierra,0.085328,0.042123,2.42,5,134.949997,2.079442,0.545455
9,58,Midnight Lion,0.626271,0.309167,16.68,1,133.949997,0.000000,0.909091


In [22]:
###############################
submission = pred_df[['Race_ID', 'Horse', 'Predicted_Probability']]
submission.to_csv('race_probabilities_main_solution.csv', index=False)

print("race_probabilities.csv written :", submission.shape)

race_probabilities.csv written : (11275, 3)
